## 0. Install dependencies

In [1]:
!pip install bs4 pandas requests lxml osmnx

Defaulting to user installation because normal site-packages is not writeable


## 1. Import requirements

In [18]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO  
import time
import random

## 2. Define constants

In [28]:
PROVINCE_LIST = ["Albay", "Camarines Norte", "Camarines Sur", "Catanduanes", "Masbate", "Sorsogon"]
PROVINCE_URL_TEMPLATE = "https://www.philatlas.com/luzon/r05/{province}.html"
LGU_URL_TEMPLATE = "https://www.philatlas.com/luzon/r05/{province}/{lgu}.html"
BARANGAY_URL_TEMPLATE = "https://www.philatlas.com/luzon/r05/{province}/{lgu}/{barangay}.html"

## 3. Get data

In [12]:
# def get_lgu_data(province: str) -> pd.DataFrame:
resp = requests.get("https://www.philatlas.com/luzon/r05/albay/bacacay.html")
soup = BeautifulSoup(resp.text, "html")

In [13]:
dfs = pd.read_html(resp.text)

/tmp/ipykernel_3795/2839089334.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(resp.text)


In [17]:
dfs[1]["Barangay"].unique().tolist()[:-1]

['Baclayon',
 'Banao',
 'Barangay 1',
 'Barangay 10',
 'Barangay 11',
 'Barangay 12',
 'Barangay 13',
 'Barangay 14',
 'Barangay 2',
 'Barangay 3',
 'Barangay 4',
 'Barangay 5',
 'Barangay 6',
 'Barangay 7',
 'Barangay 8',
 'Barangay 9',
 'Bariw',
 'Basud',
 'Bayandong',
 'Bonga',
 'Buang',
 'Busdac',
 'Cabasan',
 'Cagbulacao',
 'Cagraray',
 'Cajogutan',
 'Cawayan',
 'Damacan',
 'Gubat Ilawod',
 'Gubat Iraya',
 'Hindi',
 'Igang',
 'Langaton',
 'Manaet',
 'Mapulang Daga',
 'Mataas',
 'Misibis',
 'Nahapunan',
 'Namanday',
 'Namantao',
 'Napao',
 'Panarayon',
 'Pigcobohan',
 'Pili Ilawod',
 'Pili Iraya',
 'Pongco',
 'San Pablo',
 'San Pedro',
 'Sogod',
 'Sula',
 'Tambilagao',
 'Tambongon',
 'Tanagan',
 'Uson',
 'Vinisitahan-Basud',
 'Vinisitahan-Napao']

In [31]:
def clean_name(name):
    return name.lower().replace("/", " ").replace(" ", "-").replace("ñ", "n").replace(".", "").replace(",", "").replace("'", "").replace("(", "").replace(")", "")

request_count = 1
province_df_list = []
lgu_df_list = []
barangay_df_list = []
for province in PROVINCE_LIST:
    province_url = PROVINCE_URL_TEMPLATE.format(province=clean_name(province))
    print("Getting province:", province_url)
    province_resp = requests.get(province_url)
    province_dfs = pd.read_html(StringIO(province_resp.text), flavor="lxml")
    
    province_df = province_dfs[0].set_index(0).T
    province_df.drop(columns=province_df.columns[0], axis=1, inplace=True)

    province_df_list.append(province_df)

    print("Reqesut count: ", request_count)
    request_count +=1 

    lgu_list = province_dfs[1]["Name"].to_list()[:-1]
    for lgu in lgu_list:
        lgu_url = LGU_URL_TEMPLATE.format(province=clean_name(province), lgu=clean_name(lgu))
        print("Getting LGU", lgu_url)
        lgu_resp = requests.get(lgu_url)
        lgu_dfs = pd.read_html(StringIO(lgu_resp.text), flavor="lxml")

        lgu_df = lgu_dfs[0].set_index(0).T

        lgu_df_list.append(lgu_df)

        print("Reqesut count: ", request_count)
        request_count +=1 

        barangay_list = lgu_dfs[1]["Barangay"].to_list()[:-1]
        for barangay in barangay_list:
            barangay_url = BARANGAY_URL_TEMPLATE.format(province=clean_name(province), lgu=clean_name(lgu), barangay=clean_name(barangay))
            print("Getting brangay", barangay_url)
            barangay_resp = requests.get(barangay_url)

            barangay_dfs = pd.read_html(StringIO(barangay_resp.text), flavor="lxml")
            barangay_df = barangay_dfs[0].set_index(0).T

            barangay_df_list.append(barangay_df)

            print("Reqesut count: ", request_count)
            request_count +=1 

            sleep_time = random.uniform(1, 3)


Getting province: https://www.philatlas.com/luzon/r05/albay.html
Reqesut count:  1
Getting LGU https://www.philatlas.com/luzon/r05/albay/bacacay.html
Reqesut count:  2
Getting brangay https://www.philatlas.com/luzon/r05/albay/bacacay/baclayon.html
Reqesut count:  3
Getting brangay https://www.philatlas.com/luzon/r05/albay/bacacay/banao.html
Reqesut count:  4
Getting brangay https://www.philatlas.com/luzon/r05/albay/bacacay/barangay-1.html
Reqesut count:  5
Getting brangay https://www.philatlas.com/luzon/r05/albay/bacacay/barangay-10.html
Reqesut count:  6
Getting brangay https://www.philatlas.com/luzon/r05/albay/bacacay/barangay-11.html
Reqesut count:  7
Getting brangay https://www.philatlas.com/luzon/r05/albay/bacacay/barangay-12.html
Reqesut count:  8
Getting brangay https://www.philatlas.com/luzon/r05/albay/bacacay/barangay-13.html
Reqesut count:  9
Getting brangay https://www.philatlas.com/luzon/r05/albay/bacacay/barangay-14.html
Reqesut count:  10
Getting brangay https://www.phila

ValueError: No tables found matching regex '.+'